In [98]:
import numpy as np
import gensim.models as word2vec
model_en = word2vec.Word2Vec.load('englishwords')
model_fr = word2vec.Word2Vec.load('englishwords')

In [99]:
lines_en = open('test.en','r',encoding='utf8').readlines()

In [100]:
data_en = open('test.en','r',encoding='utf8').read()
data_en = data_en.split()

In [101]:
words_en = list(set(data_en))
data_size_en,vocab_size_en = len(data_en),len(model_en.wv['the'])
print('english file has %d words,%d features'%(data_size_en,vocab_size_en))

word_to_ix_en = {}
ix_to_word_en = {}

english file has 252583 words,300 features


In [102]:
for w in words_en:
    word_to_ix_en[w] = model_en.wv[w]
    ix_to_word_en[tuple(model_en.wv[w])] = w
print('Word2Vec English unique words = %d, word2vector features = %d'%(len(word_to_ix_en),len(model_en.wv['the'])))

Word2Vec English unique words = 19176, word2vector features = 300


In [103]:
lines_fr = open('tester.en','r',encoding='utf8').readlines()
data_fr = open('tester.en','r',encoding='utf8').read()
data_fr = data_fr.split()
words_fr = list(set(data_fr))
data_size_fr,vocab_size_fr = len(data_fr),len(model_en.wv['la'])
print('french file has %d words,%d features'%(data_size_fr,vocab_size_fr))
word_to_ix_fr = {}
ix_to_word_fr = {}

french file has 254 words,300 features


In [104]:
for w in words_fr:
    word_to_ix_fr[w] = model_fr.wv[w]
    ix_to_word_fr[tuple(model_fr.wv[w])] = w
print('Word2Vec French unique words =',len(word_to_ix_fr))

Word2Vec French unique words = 149


In [105]:
num = len(lines_en)
num = len(lines_fr) #No of Sentences


In [106]:
#hyperparameters which are same for both encoder and decoder
hidden_size = 100
learning_rate = 1e-1


In [107]:
#encoder weight parameters for english language
wxh_en = np.random.randn(hidden_size,vocab_size_en)*0.01
whh_en = np.random.randn(hidden_size,hidden_size)*0.01
why_en = np.random.randn(vocab_size_en,hidden_size)*0.01
bh_en = np.zeros((hidden_size,1))
by_en = np.zeros((vocab_size_en,1))

In [108]:
#decoder weight parameters for french language
wxh_fr = np.random.randn(hidden_size,vocab_size_fr)*0.01
whh_fr = np.random.randn(hidden_size,hidden_size)*0.01
why_fr = np.random.randn(vocab_size_fr,hidden_size)*0.01
bh_fr = np.zeros((hidden_size,1))
by_fr = np.zeros((vocab_size_fr,1))

In [109]:
def trainencoder(inputs, targets, hprev):
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  # forward pass  
#   print(len(inputs))
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size_en,1)) # encode in 1-of-k representation
    a = np.reshape(inputs[t],(300,1))
    xs[t] = np.copy(a)
    hs[t] = np.tanh(np.dot(wxh_en, xs[t]) + np.dot(whh_en, hs[t-1]) + bh_en) # hidden state
    ys[t] = np.dot(why_en, hs[t]) + by_en # unnormalized log probabilities for next words
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next words
#   print(hs)
  dWxh, dWhh, dWhy = np.zeros_like(wxh_en), np.zeros_like(whh_en), np.zeros_like(why_en)
  dbh, dby = np.zeros_like(bh_en), np.zeros_like(by_en)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    target = np.reshape(targets[t],(300,1))
    dy = np.copy(ys[t])
    dy -= target  # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(why_en.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(whh_en.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [110]:
def traindecoder(inputs, targets, hprev):
  xs, hs, ys, ps = {}, {}, {}, {}
  hs[-1] = np.copy(hprev)
  # forward pass
  for t in range(len(inputs)):
    xs[t] = np.zeros((vocab_size_fr,1)) # encode in 1-of-k representation
    if(int(inputs[0])!=-1):
        a = np.reshape(inputs[t],(300,1))
        xs[t] = np.copy(a)
    hs[t] = np.tanh(np.dot(wxh_fr, xs[t]) + np.dot(whh_fr, hs[t-1]) + bh_fr) # hidden state
    ys[t] = np.dot(why_fr, hs[t]) + by_fr # unnormalized log probabilities for next chars
    ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars

  dWxh, dWhh, dWhy = np.zeros_like(wxh_fr), np.zeros_like(whh_fr), np.zeros_like(why_fr)
  dbh, dby = np.zeros_like(bh_fr), np.zeros_like(by_fr)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(range(len(inputs))):
    target = np.reshape(targets[t],(300,1))
    dy = np.copy(ys[t])
    dy -= target # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(why_fr.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(whh_fr.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [111]:
def test(inputs, targets):
  xs, hs, ys = {}, {}, {}
  hs[-1] = np.zeros((hidden_size,1))
  # forward pass
  for t in range(len(inputs)):
    a = np.reshape(inputs[t],(300,1))
    xs[t] = np.zeros((vocab_size_en,1)) # encode in 1-of-k representation
    xs[t] = np.copy(a)
    hs[t] = np.tanh(np.dot(wxh_en, xs[t]) + np.dot(whh_en, hs[t-1]) + bh_en) # hidden state
    ys[t] = np.dot(why_en, hs[t]) + by_en 
  hprev = hs[len(inputs)-1]
  tem = ""
  ans = ""
  k = 0
  t = -1
  while k<10:
      x = np.zeros((vocab_size_fr,1))
      if(t!=-1):
          x[t] = 1
      hprev = np.tanh(np.dot(wxh_fr, x) + np.dot(whh_fr, hprev) + bh_fr)
      y = np.dot(why_fr, hprev) + by_fr # unnormalized log probabilities for next chars
      keys = np.zeros((len(ix_to_word_fr)))
      mind = 100000.0
      min_key = tuple
      for ix in ix_to_word_fr.keys():
          cosdist = np.dot(np.asarray(ix).T,y)/(np.linalg.norm(np.asarray(ix))*np.linalg.norm(y))
          if(cosdist<mind):
                mind = cosdist
                minkey = ix
#       print(keys.shape,y.shape)
#       print(ix_to_word_fr[minkey],mind)
      tem = ix_to_word_fr[minkey]
#       pr = np.exp(y) / np.sum(np.exp(y))
#       print(y.shape,y.reshape(300).shape)
#       maxi = y[0][0]
#       for i in range(len(words_fr)):
#           if y[i][0]>=maxi:
#               maxi = y[i][0]
#               t = i
#       arr = np.asarray(ix_to_word_fr);
#       print(arr[np.linalg.norm(arr-y, axis=1).argmin()]) # closest key to y vector is the word
      #tem = ix_to_word_fr[tuple(y.reshape(300))] # J Put word in it. find the closest word to y(vector)
      k = k + 1
      ans = ans + " " +tem
  return ans

In [112]:
n,p = 1,0
mwxh_en,mwhh_en,mwhy_en,mbh_en,mby_en = np.zeros_like(wxh_en),np.zeros_like(whh_en),np.zeros_like(why_en),np.zeros_like(bh_en),np.zeros_like(by_en)
mwxh_fr,mwhh_fr,mwhy_fr,mbh_fr,mby_fr = np.zeros_like(wxh_fr),np.zeros_like(whh_fr),np.zeros_like(why_fr),np.zeros_like(bh_fr),np.zeros_like(by_fr)
print(n,p)

1 0


In [114]:
while n!=2000:
    curr_en = lines_en[p].split()
    inputs_en = [word_to_ix_en[w] for w in curr_en[0:len(curr_en)-1]]
    targets_en = [word_to_ix_en[w] for w in curr_en[1:len(curr_en)]]
    
    curr_fr = lines_fr[p].split()
    inputs_fr=[-1]
    temp = [word_to_ix_fr[w] for w in curr_fr[0:len(curr_fr)-1]]
    inputs_fr.extend(temp)
    targets_fr = [word_to_ix_fr[w] for w in curr_fr[0:len(curr_fr)]]

    hprev = np.zeros((hidden_size,1))
#     print("\bTraining Encoder",n,p)
    dwxh_en,dwhh_en,dwhy_en,dbh_en,dby_en,hprev = trainencoder(inputs_en,targets_en,hprev)
#     print("\bTraining Decoder",n,p)
    dwxh_fr,dwhh_fr,dwhy_fr,dbh_fr,dby_fr,hprev = traindecoder(inputs_fr,targets_fr,hprev)
    p += 1 # move sentence pointer
    if p >= num:
        p = 0
        print('\b>>Training...iteration:%d'%(n))
        input_english = "in that context."
        curr_en = input_english.split()
        inputs_en = [word_to_ix_en[w] for w in curr_en[0:len(curr_en)-1]]
        targets_en = [word_to_ix_en[w] for w in curr_en[1:len(curr_en)]]
        output_words = test(inputs_en,targets_en)
        print(output_words)  
        n = n + 1
    for param_en, dparam_en, mem_en in zip([wxh_en, whh_en, why_en, bh_en, by_en], 
                                [dwxh_en, dwhh_en, dwhy_en, dbh_en, dby_en], 
                                [mwxh_en, mwhh_en, mwhy_en, mbh_en, mby_en]):
      mem_en += dparam_en * dparam_en
      param_en += -learning_rate * dparam_en / np.sqrt(mem_en + 1e-8) # adagrad update
    





    for param_fr, dparam_fr, mem_fr in zip([wxh_fr, whh_fr, why_fr, bh_fr, by_fr], 
                                [dwxh_fr, dwhh_fr, dwhy_fr, dbh_fr, dby_fr], 
                                [mwxh_fr, mwhh_fr, mwhy_fr, mbh_fr, mby_fr]):
      mem_fr += dparam_fr * dparam_fr
      param_fr += -learning_rate * dparam_fr / np.sqrt(mem_fr + 1e-8) # adagrad update
 # iteration counter 

>>Training...iteration:100
 Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:101
 Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:102
 Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:103
 Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:104
 Disaster Disaster Disaster ACC Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:105
 (ICSC) Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:106
 (ICSC) Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:107
 (ICSC) Disaster Disaster ACC Disaster Disaster Disaster Disaster Disaster Disaster
>>Training...iteration:108
 (ICSC) Strategy Disaster ACC Disast